In [5]:
import os, random, shutil
import numpy as np
import pandas as pd
import PIL
#import keras
import itertools
from PIL import Image

import tensorflow as tf

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
#from skimage import feature, data, io, measure
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 
from tqdm import tqdm
import cv2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.layers import BatchNormalization
#from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import TensorBoard
import io
import time
import datetime
from tensorflow.keras.optimizers import Adam,Adadelta
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import urllib

## Model building

In [2]:
batch_size_train = 25
num_classes= 4
IMAGE_SIZE=[200,200]
classes_required = ['Aviator','Oval','Rectangle','Wayfarer']

In [3]:
datagen = ImageDataGenerator(rescale=1.0/255.0,
                             #shear_range = 0.2,
                             #zoom_range = 0.2
                            )

In [4]:
#Input the Training Data

train_path = 'D:/My ML Projects/lenskart task/sub-train/sunglasses'
train_batches = datagen.flow_from_directory(train_path, target_size=(200,200), classes=classes_required, batch_size=batch_size_train)
type(train_batches)

Found 3367 images belonging to 4 classes.


tensorflow.python.keras.preprocessing.image.DirectoryIterator

In [6]:
batchX, batchy = train_batches.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Batch shape=(25, 200, 200, 3), min=0.000, max=1.000


In [7]:
train_imgs, train_labels = next(train_batches)

In [8]:
train_imgs[0]

array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       ...,

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]], dtype=float32)

In [9]:
train_imgs[0]

array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       ...,

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]], dtype=float32)

In [10]:
config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 64} ) 
sess = tf.compat.v1.Session(config=config) 
#keras.backend.set_session(sess)
tf.compat.v1.keras.backend.set_session(
    sess
)
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True, per_process_gpu_memory_fraction=0.333)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

### Model Architecture

In [11]:
model=Sequential()
model.add(Conv2D(16,kernel_size=(3,3), activation="relu" ,input_shape=IMAGE_SIZE + [3], padding='same'))

model.add(Conv2D(32, kernel_size=(3,3), activation="relu",padding='same'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.30))

model.add(Conv2D(64, kernel_size=(3,3), activation="relu",padding='same'))
#model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.35))

model.add(Conv2D(128, kernel_size=(3,3), activation="relu",padding='same'))
#model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization()) 
model.add(Dropout(0.45))

model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dense(num_classes, activation="softmax"))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 200, 16)      448       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 200, 200, 32)      4640      
_________________________________________________________________
batch_normalization (BatchNo (None, 200, 200, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 100, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 64)        0

In [13]:
# Complie the model
opt = Adadelta(learning_rate=0.0001, decay=1e-6)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.fit_generator(train_batches, steps_per_epoch=25, epochs=100, verbose=1)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
25/25 [==============================] - 8s 319ms/step - loss: 2.2891 - accuracy: 0.2384
Epoch 2/100
25/25 [==============================] - 7s 298ms/step - loss: 2.1947 - accuracy: 0.2656
Epoch 3/100
25/25 [==============================] - 8s 301ms/step - loss: 2.1337 - accuracy: 0.2512
Epoch 4/100
25/25 [==============================] - 7s 299ms/step - loss: 2.0554 - accuracy: 0.2688
Epoch 5/100
25/25 [==============================] - 7s 285ms/step - loss: 2.0361 - accuracy: 0.2864
Epoch 6/100
25/25 [==============================] - 7s 289ms/step - loss: 1.9841 - accuracy: 0.2736
Epoch 7/100
25/25 [==============================] - 7s 272ms/step - loss: 1.8337 - accuracy: 0.3040
Epoch 8/100
25/25 [==============================] - 7s 276ms/step - loss: 1.9114 - accuracy: 0.2784
Epoch 9/100
25/25 [==============================] - 6s 256ms/step - loss: 1.7947 - accuracy: 0.3152
Epoch 10/100
25

25/25 [==============================] - 8s 308ms/step - loss: 1.4077 - accuracy: 0.4032
Epoch 77/100
25/25 [==============================] - 8s 304ms/step - loss: 1.3501 - accuracy: 0.4344
Epoch 78/100
25/25 [==============================] - 8s 322ms/step - loss: 1.2621 - accuracy: 0.4672
Epoch 79/100
25/25 [==============================] - 8s 311ms/step - loss: 1.3071 - accuracy: 0.4416
Epoch 80/100
25/25 [==============================] - 8s 316ms/step - loss: 1.3219 - accuracy: 0.4489
Epoch 81/100
25/25 [==============================] - 8s 312ms/step - loss: 1.2342 - accuracy: 0.4765
Epoch 82/100
25/25 [==============================] - 8s 317ms/step - loss: 1.3067 - accuracy: 0.4384
Epoch 83/100
25/25 [==============================] - 8s 317ms/step - loss: 1.3391 - accuracy: 0.4000
Epoch 84/100
25/25 [==============================] - 8s 306ms/step - loss: 1.2521 - accuracy: 0.4688
Epoch 85/100
25/25 [==============================] - 8s 325ms/step - loss: 1.3275 - accuracy: 

In [15]:
model.save("model_Sunglass_25_batch.h5")
print("Saved model to disk")

Saved model to disk
